In [1]:
# Operações de Crédito - Conta 16000001 (filtragem maiores saldos)

import pandas as pd
import sqlite3

conn = sqlite3.connect('dados/banking.db')

query = """
WITH RankedData AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY data ORDER BY "SALDO" DESC) AS rn
    FROM
        balancetes
    WHERE
        "CONTA" = '16000001'
)
SELECT
    *
FROM
    RankedData
WHERE
    SALDO > 1000000
--    rn <= 10;
"""

model = pd.read_sql_query(query, conn)

conn.close()

model = model[['data', 'cnpj', 'NOME_INSTITUICAO', 'SALDO']]
model['cnpj'] = model['cnpj'].astype(int)
model = model.rename(columns={'SALDO': 'operacoes_de_credito'})


In [2]:
# Ativo circulante
'''
11000006	DISPONIBILIDADES
12000005	APLICACOES INTERFINANCEIRAS DE LIQUIDEZ
13100007	Livres
14000003	RELACOES INTERFINANCEIRAS
15000002	RELACOES INTERDEPENDENCIAS
16000001	OPERACOES DE CREDITO
18000009	OUTROS CREDITOS
19900005	Despesas Antecipadas
'''
import numpy as np

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('11000006', '12000005', '13100007', '14000003', '15000002', '16000001', '18000009', '19900005')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_circulante'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [3]:
# Passivo circulante
'''
40000008	CIRCULANTE E EXIGIVEL A LONGO PRAZO
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('40000008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'passivo_circulante'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [4]:
# Patrimônio Liquido
'''
60000002	PATRIMONIO LIQUIDO
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('60000002')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'patrimonio_liquido'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [5]:
# SHORT

model['short'] = (model['ativo_circulante'] - model['passivo_circulante']) / model['patrimonio_liquido']



In [6]:
# Provisões de crédito
'''
16900008	(-) Provisoes Para Operacoes De Credito
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('16900008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table['saldo_total_agregado'] = -table['saldo_total_agregado']
table = table.rename(columns={'saldo_total_agregado': 'provisao_credito'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [7]:
# NCO

model['nco'] = model['provisao_credito'] / model['operacoes_de_credito']



In [8]:
# NCO * SHORT

model['nco*short'] = model['nco'] * model['short']


In [9]:
# Ativos liquidos (Caixa e Equivalentes de Caixa)
'''
Caixa (1.1.1.00.00-9)
Depósitos Bancários (1.1.2.00.00-2)
Reservas Livres (1.1.3.00.00-5)
Aplicações em Operações Compromissadas (1.2.1.00.00-8)
Aplicações em Depósitos Interfinanceiros (1.2.2.00.00-1)
Disponibilidades em Moedas Estrangeiras (1.1.5.00.00-1)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('11100009', '11200002', '11300005', '12100008', '12200001', '11500001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_liquido'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [10]:
# LIQ

model['liq'] = model['ativo_liquido'] / model['patrimonio_liquido']



In [11]:
# Core Capital
'''
61100004	Capital Social
61500006	Reservas De Lucros
61800005	Lucros Ou Prejuizos Acumulados
61600009	Ajustes De Avaliacao Patrimonial
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('61100004', '61500006', '61800005', '61600009')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()


table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'core_capital'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)



In [12]:
# Ativo Total
'''
39999993	TOTAL GERAL DO ATIVO
'''
import numpy as np

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('39999993')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_total'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [13]:
# Leverage

model['lev'] = model['core_capital'] / model['ativo_total']


In [14]:
# Receita não financeira
'''
71700009	Rendas De Prestacao De Servicos
71900005	Outras Receitas Operacionais
73000006	RECEITAS NAO OPERACIONAIS
'''
import numpy as np

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
        "CONTA" IN ('71700009', '73000006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_nao_financeira'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [15]:
# Despesa não financeira
'''
81700006	(-) Despesas Administrativas
81900002	(-) Outras Despesas Operacionais
83000003	(-) DESPESAS NAO OPERACIONAIS
'''
import numpy as np

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
        "CONTA" IN ('71700009', '73000006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'despesa_nao_financeira'})
table['despesa_nao_financeira'] = -table['despesa_nao_financeira']

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [16]:
# Ativos Rentáveis
'''
Operações de Crédito (1.6.0.00.00-1)
Títulos e Valores Mobiliários (1.3.0.00.00-4)
Arrendamento Mercantil (1.7.0.00.00-1)
12000005	APLICACOES INTERFINANCEIRAS DE LIQUIDEZ
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('16000001', '13000004', '17000001', '12000005')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativos_rentaveis'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [17]:
# IMPLICIT - Implicit Interest Payments

model['implicit'] = (model['despesa_nao_financeira'] - model['receita_nao_financeira']) / model['ativos_rentaveis']


In [18]:
# Non Interest Bearing Reserve Asset
'''
CAIXA (1.1.1.00.00-9)
DEPÓSITOS BANCÁRIOS (1.1.2.00.00-2)
Reservas Livres (1.1.3.00.00-5)
'''
conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('11100009', '11200002', '11300005')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)

conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_reserva_nao_remunerada'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [19]:
# NIBR - Non Interest Bearing Reserves

model['nibr'] = model['ativo_reserva_nao_remunerada'] / model['ativo_total']


In [20]:
# MGMT - Management Efficiency

model['mgmt'] = model['ativos_rentaveis'] / model['ativo_total']

In [21]:
# Receitas de Juros e Similares (Rendas)
'''
Rendas De Operacoes De Credito (7.1.1.00.00-1)
Rendas de Arrendamento Mercantil (7.1.2.00.00-4)
Rendas de Aplicações Interfinanceiras de Liquidez (7.1.4.00.00-0)
Rendas de Títulos e Valores Mobiliários (7.1.5.00.00-3)
Rendas de Outras Operações Com Características de Crédito (7.1.6.00.00-8)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('71100001', '71200004', '71400000', '71500003', '71600008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_juros'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [ ]:
# Despesas de Juros e Encargos (Custos de Captação)
'''
Despesas de captação (81100008)
Despesas de Obrigações por Empréstimos e Repasses (8.1.2.00.00-1)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81100008', '81200001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'despesa_captacao'})
table['despesa_captacao'] = table['despesa_captacao'] * (-1)

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

model['log_despesa_captacao'] = np.log10(model['despesa_captacao'])

In [23]:
# NIM

model['nim'] = 100 * (model['receita_juros'] - model['despesa_captacao']) / model['ativos_rentaveis']

In [24]:
# Criando tabela variables

variables = model[['data', 'cnpj', 'NOME_INSTITUICAO', 'nim', 'nco', 'short','nco*short', 'liq', 'lev', 'implicit', 'nibr', 'mgmt']]

# Variáveis lags
'''
coluna_grupo = 'cnpj'
colunas_para_verificar = ['nibra', 'nibd', 'ibf', 'lever', 'opc', 'liquid', 'servr', 'mktsh']

for col in colunas_para_verificar:
    variables[f'{col}_lag_2'] = (variables.groupby(coluna_grupo)[col].shift(2))
    variables[f'{col}_lag_1'] = (variables.groupby(coluna_grupo)[col].shift(1))
    variables[f'{col}_lag_3'] = (variables.groupby(coluna_grupo)[col].shift(3))
'''
# Dummies de tempo
dummies_tempo = pd.get_dummies(variables['data'], drop_first=True)
dummies_tempo = dummies_tempo.astype(int)
variables = pd.concat([variables, dummies_tempo], axis=1)

# Removendo linhas com valores nulos, exceto na coluna NOME_INSTITUICAO
todas_colunas = variables.columns.tolist()
colunas_para_verificar = [col for col in todas_colunas if col != 'NOME_INSTITUICAO']
variables = variables.dropna(how='any', subset=colunas_para_verificar)


In [26]:
# Tratando outliers usando o método do IQR (Interquartile Range)
colunas_para_analisar = ['nim', 'nco', 'short','nco*short', 'liq', 'lev', 'implicit', 'nibr', 'mgmt']

# True significa que a linha *será removida* (outlier).
# Começamos com False, assumindo que nenhuma linha será removida inicialmente.
mascara_outliers_acumulada = pd.Series(False, index=variables.index)

print(f"Tamanho do DataFrame antes da remoção: {len(variables)}")

for col in colunas_para_analisar:
    Q1 = variables[col].quantile(0.10)
    Q3 = variables[col].quantile(0.90)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    mascara_outliers_coluna = (variables[col] < limite_inferior) | (variables[col] > limite_superior)

    mascara_outliers_acumulada = mascara_outliers_acumulada | mascara_outliers_coluna

    num_outliers_col = mascara_outliers_coluna.sum()
    print(f"Coluna '{col}': {num_outliers_col} outliers identificados (Limites: [{limite_inferior:.2f}, {limite_superior:.2f}])")


# Criamos uma máscara de linhas *a serem mantidas* (True = não é outlier)
mascara_a_manter = ~mascara_outliers_acumulada # O til (~) inverte a máscara
variables_limpo = variables[mascara_a_manter]

print(f"\nTotal de linhas removidas: {mascara_outliers_acumulada.sum()}")
print(f"Tamanho do DataFrame após a remoção: {len(variables_limpo)}")

Tamanho do DataFrame antes da remoção: 134648
Coluna 'nim': 3589 outliers identificados (Limites: [-13.37, 24.34])
Coluna 'nco': 4407 outliers identificados (Limites: [-0.16, 0.31])
Coluna 'short': 5652 outliers identificados (Limites: [-0.80, 2.06])
Coluna 'nco*short': 6253 outliers identificados (Limites: [-0.13, 0.23])
Coluna 'liq': 5748 outliers identificados (Limites: [-1.59, 2.67])
Coluna 'lev': 2671 outliers identificados (Limites: [-0.19, 0.38])
Coluna 'implicit': 2826 outliers identificados (Limites: [-0.14, 0.08])
Coluna 'nibr': 3306 outliers identificados (Limites: [-0.01, 0.02])
Coluna 'mgmt': 171 outliers identificados (Limites: [-0.32, 0.83])

Total de linhas removidas: 21536
Tamanho do DataFrame após a remoção: 113112


In [28]:
# Modelo - Primeiro Estágio
import statsmodels.api as sm

colunas_X_originais = ['nco', 'short', 'liq', 'lev', 'implicit', 'nibr', 'mgmt']
colunas_X = colunas_X_originais + list(dummies_tempo.columns)
X = variables_limpo[colunas_X]
y = variables_limpo['nim']

stage1 = sm.OLS(y, sm.add_constant(X)).fit()
results_stage1 = stage1
print(results_stage1.summary())

spread_estimado = variables_limpo[['data', 'cnpj', 'NOME_INSTITUICAO']]
spread_estimado['spread_estimado'] = results_stage1.predict()
spread_estimado['spread_puro'] = results_stage1.params['const'] + variables_limpo[dummies_tempo.columns].dot(results_stage1.params[dummies_tempo.columns])


                            OLS Regression Results                            
Dep. Variable:                    nim   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     495.7
Date:                Sat, 08 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:43:42   Log-Likelihood:            -2.6716e+05
No. Observations:              113112   AIC:                         5.347e+05
Df Residuals:                  112925   BIC:                         5.365e+05
Df Model:                         186                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1559      0.220      0.707      0.4

/tmp/ipykernel_601551/2295521023.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spread_estimado['spread_estimado'] = results_stage1.predict()
/tmp/ipykernel_601551/2295521023.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spread_estimado['spread_puro'] = results_stage1.params['const'] + variables_limpo[dummies_tempo.columns].dot(results_stage1.params[dummies_tempo.columns])
